In [ ]:
import cv2
from pathlib import Path


def crop_to_mask(image_path, output_path):
    # 이미지 로드
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # 검은색 배경을 기준으로 마스크 생성
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # 마스크에서 경계 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"No contours found in image: {image_path}")
        return

    # 가장 큰 경계에 해당하는 영역을 찾아 크롭
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    cropped = image[y : y + h, x : x + w]

    # 크롭된 이미지 저장
    cv2.imwrite(str(output_path), cropped)


# 입력 및 출력 디렉토리 설정
input_dir = Path(
    "/content/drive/MyDrive/비타민-2024-2-CV/Before_Crop_Images"
)
output_dir = Path(
    "/content/drive/MyDrive/비타민-2024-2-CV/After_Crop_Images"
)

# 입력 디렉토리의 모든 이미지에 대해 반복
for category in ["test", "train", "valid"]:
    for emotion in ["Angry", "Happy", "Neutral", "Sad", "Surprised"]:
        current_dir = input_dir / category / emotion
        output_current_dir = output_dir / category / emotion
        output_current_dir.mkdir(parents=True, exist_ok=True)  # 출력 디렉토리 생성

        for image_path in current_dir.glob("*.jpg"):
            output_image_path = output_current_dir / image_path.name
            crop_to_mask(image_path, output_image_path)